# Sales Performance Dashboard Analysis
## Data Science Assignment

This notebook demonstrates the creation of a comprehensive sales performance dashboard using the Online Retail Dataset. We'll follow data visualization best practices and implement the 4C Principles:
- Clear
- Concise
- Captivating
- Credible

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn')
sns.set_palette('Set2')

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## 1. Data Loading and Cleaning

In [3]:
# Load the dataset
df = pd.read_excel('Online Retail Data Set.xlsx')

# Display basic information about the dataset
print("Dataset Info:")
print(df.info())
print("\nFirst few rows:")
df.head()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None

First few rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Data cleaning steps
def clean_data(df):
    # Remove rows with missing values
    df = df.dropna()
    
    # Remove rows with negative quantities or prices
    df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
    
    # Add TotalAmount column
    df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
    
    # Convert InvoiceDate to datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    
    # Extract month and year
    df['Month'] = df['InvoiceDate'].dt.to_period('M')
    
    return df

# Clean the data
df_clean = clean_data(df.copy())
print("Shape after cleaning:", df_clean.shape)

Shape after cleaning: (397884, 10)


## 2. Monthly Sales Trends

In [ ]:
# Calculate monthly sales
monthly_sales = df_clean.groupby('Month')['TotalAmount'].sum().reset_index()
monthly_sales['Month'] = monthly_sales['Month'].astype(str)

# Create monthly sales trend visualization
fig = px.line(monthly_sales, x='Month', y='TotalAmount',
              title='Monthly Sales Trends',
              labels={'TotalAmount': 'Total Sales', 'Month': 'Month'},
              template='plotly_white')
fig.update_layout(showlegend=False)
fig.show()

## 3. Best-Selling Products Analysis

In [ ]:
# Calculate top 10 best-selling products by quantity
top_products = df_clean.groupby('Description')[
    ['Quantity', 'TotalAmount']
].agg({
    'Quantity': 'sum',
    'TotalAmount': 'sum'
}).sort_values('TotalAmount', ascending=False).head(10)

# Create visualization for best-selling products
fig = px.bar(top_products, y=top_products.index, x='TotalAmount',
             title='Top 10 Best-Selling Products by Revenue',
             labels={'index': 'Product', 'TotalAmount': 'Total Revenue'},
             orientation='h',
             template='plotly_white')
fig.show()

## 4. Sales by Country

In [ ]:
# Calculate sales by country
country_sales = df_clean.groupby('Country')['TotalAmount'].sum().sort_values(ascending=True)

# Create visualization for country sales
fig = px.bar(country_sales, 
             x=country_sales.values,
             y=country_sales.index,
             title='Sales by Country',
             labels={'y': 'Country', 'x': 'Total Sales'},
             template='plotly_white')
fig.show()

## 5. Product Categories Analysis

In [ ]:
# Function to extract categories from description
def extract_category(description):
    # Common categories in retail data
    categories = {
        'VINTAGE': 'Vintage Items',
        'GARDEN': 'Garden Accessories',
        'CHRISTMAS': 'Christmas Items',
        'METAL': 'Metal Signs & Decor',
        'WOOD': 'Wooden Items',
        'GLASS': 'Glassware',
        'PAPER': 'Paper Products',
        'CERAMIC': 'Ceramic Items'
    }
    
    description = str(description).upper()
    for key in categories:
        if key in description:
            return categories[key]
    return 'Other'

# Add category column
df_clean['Category'] = df_clean['Description'].apply(extract_category)

# Analyze sales by category
category_sales = df_clean.groupby('Category')['TotalAmount'].sum().sort_values(ascending=True)

# Create visualization for category sales
fig = px.pie(values=category_sales.values,
             names=category_sales.index,
             title='Sales Distribution by Product Category',
             template='plotly_white')
fig.show()

## 6. Customer Behavior Analysis

In [ ]:
# Create customer cohorts
def create_cohort(df):
    # Get the first purchase date for each customer
    df['CohortMonth'] = df.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')
    
    # Calculate cohort index
    df['CohortIndex'] = (df['InvoiceDate'].dt.to_period('M') - 
                         df['CohortMonth']).apply(lambda x: x.n)
    
    return df

# Create cohorts
df_cohort = create_cohort(df_clean[df_clean['CustomerID'].notna()])

# Create cohort analysis
cohort_data = df_cohort.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].nunique().reset_index()
cohort_table = cohort_data.pivot(index='CohortMonth',
                                columns='CohortIndex',
                                values='CustomerID')

# Calculate retention rates
cohort_sizes = cohort_table.iloc[:, 0]
retention_table = cohort_table.divide(cohort_sizes, axis=0)

# Plot retention heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(retention_table,
            annot=True,
            fmt='.0%',
            cmap='YlOrRd')
plt.title('Customer Cohort Retention Analysis')
plt.xlabel('Cohort Index (Months)')
plt.ylabel('Cohort Month')
plt.show()

## 7. RFM Analysis

In [ ]:
def calculate_rfm(df):
    # Calculate Recency, Frequency, Monetary metrics
    latest_date = df['InvoiceDate'].max()
    rfm = df.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (latest_date - x.max()).days,  # Recency
        'InvoiceNo': 'count',  # Frequency
        'TotalAmount': 'sum'  # Monetary
    })
    
    rfm.columns = ['Recency', 'Frequency', 'Monetary']
    
    # Create RFM scores
    r_labels = range(4, 0, -1)
    r_quartiles = pd.qcut(rfm['Recency'], q=4, labels=r_labels)
    f_labels = range(1, 5)
    f_quartiles = pd.qcut(rfm['Frequency'], q=4, labels=f_labels)
    m_labels = range(1, 5)
    m_quartiles = pd.qcut(rfm['Monetary'], q=4, labels=m_labels)
    
    rfm['R'] = r_quartiles
    rfm['F'] = f_quartiles
    rfm['M'] = m_quartiles
    
    return rfm

# Calculate RFM metrics
rfm_df = calculate_rfm(df_clean[df_clean['CustomerID'].notna()])

# Create RFM segments
rfm_df['RFM_Score'] = rfm_df['R'].astype(str) + \
                      rfm_df['F'].astype(str) + \
                      rfm_df['M'].astype(str)

# Visualize RFM distribution
fig = px.scatter_3d(rfm_df, x='Recency', y='Frequency', z='Monetary',
                    color='RFM_Score',
                    title='3D RFM Analysis',
                    template='plotly_white')
fig.show()

## 8. Product Associations

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Create basket data
basket = df_clean.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)
basket_sets = (basket > 0).astype(int)

# Generate frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules = rules.sort_values('lift', ascending=False)

# Visualize top 10 association rules
plt.figure(figsize=(10, 6))
sns.scatterplot(data=rules.head(10), x='support', y='confidence', size='lift', hue='lift')
plt.title('Top 10 Product Association Rules')
plt.show()

## Summary of Analysis

1. **Monthly Sales Trends**:
   - Visualized temporal patterns in sales
   - Identified seasonal trends and peak sales periods

2. **Best-Selling Products**:
   - Analyzed top 10 products by revenue
   - Highlighted key revenue-generating items

3. **Sales by Country**:
   - Mapped global sales distribution
   - Identified key markets and opportunities

4. **Product Categories**:
   - Extracted and analyzed product categories
   - Visualized sales distribution across categories

5. **Customer Behavior**:
   - Created customer cohorts
   - Analyzed retention patterns

6. **RFM Analysis**:
   - Segmented customers based on Recency, Frequency, and Monetary value
   - Identified high-value customer segments

7. **Product Associations**:
   - Discovered frequently co-purchased items
   - Identified strong product relationships

This dashboard provides a comprehensive view of the business performance, customer behavior, and product relationships, enabling data-driven decision-making for the management team.